In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sbn

from sklearn.metrics import f1_score
from matplotlib import pyplot as plt

from other_paper.balancers  import BinaryBalancer, MulticlassBalancer
from other_paper import tools

from fmclp.main import fmclp
from sklearn.model_selection import train_test_split

from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

from fmclp.cuae_metric import cuae

### GPA

In [2]:
gpa = pd.read_csv('bar_pass_prediction.csv')

del gpa['ID']
del gpa['race1']
del gpa['race2']
del gpa['sex']
del gpa['bar']
del gpa['dnn_bar_pass_prediction']
del gpa['pass_bar']
del gpa['indxgrp2']
del gpa['gender']
del gpa['grad']
del gpa['Dropout']
del gpa['fulltime']
del gpa['lsat']
del gpa['zfygpa']
del gpa['ugpa']
del gpa['zgpa']
del gpa['other']
del gpa['asian']
del gpa['black']
del gpa['hisp']

def grouper_race(x):
    if x == 7:
        return 1
    else: 
        return 0
    
gpa['race'] = gpa['race'].apply(grouper_race)

def grouper_gpa(x):
    if x>3.4:
        return 2
    elif x <3.1:
        return 0
    else:
        return 1

gpa['gpa'] = gpa['gpa'].apply(grouper_gpa)
gpa = gpa.rename(columns = {'gpa':'target', 'race':'attr'})
gpa = pd.get_dummies(gpa,drop_first = True)
gpa = gpa.dropna(how = 'any')

In [3]:
s=1
num=100
metric_list = []
accuracy_list = []
diff_list = []
ratio_list = []
variation_list = []

for i in range(num): 
    
    y = gpa.drop(['target'], axis=1) 
    x = gpa['target']
    y_train,y_test,x_train,x_test = train_test_split(y,x)
    gpa_classif = LGBMClassifier()
    gpa_classif.fit(y_train, x_train)
    gpa_preds = gpa_classif.predict(y_test)
    
    a = y_train['attr'].values
    y = x_train.values
    y_ = np.array(gpa_classif.predict(y_train))
        
    gpa_balancer = MulticlassBalancer(y=y, y_=y_, a=a)
    gpa_balancer.adjust(goal='odds')
    gpa_preds = gpa_balancer.predict(y_ = np.array(gpa_classif.predict(y_test)),a = y_test['attr'].values)
    
    accuracy = accuracy_score(gpa_preds, x_test)
    metric = cuae(y_true = x_test, y_pred=gpa_preds, sensitive_features=y_test['attr'].values)
    
    accuracy_list.append(accuracy)
    metric_list.append(metric)
    diff_list.append(metric['diff'])
    ratio_list.append(metric['ratio'])
    variation_list.append(metric['variation'])

    print(s)
    s+=1
    
accuracy_list = np.array(accuracy_list)
diff_list = np.array(diff_list)
ratio_list = np.array(ratio_list)
variation_list = np.array(variation_list)

accuracy_mean = accuracy_list.mean()
diff_mean = diff_list.mean()
ratio_mean = ratio_list.mean()
variation_mean = variation_list.mean()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [4]:
file = open("comparison_results/gpa.txt", 'w') 
file.write(f"""accuracy: {accuracy_list}
diff: {diff_list}
ratio: {ratio_list}
var: {variation_list}

accuracy_mean: {accuracy_list.mean()}
diff_mean: {diff_list.mean()}
ratio_mean: {ratio_list.mean()}
var_mean: {variation_list.mean()}
""")
file.close()

In [6]:
print(f"""average_accuracy: {accuracy_list[:10].mean()}
average_diff: {diff_list[:10].mean()}
average_ratio: {ratio_list[:10].mean()}
average_variation: {variation_list[:10].mean()}""")

average_accuracy: 0.6589582925396514
average_diff: 0.16102017240348598
average_ratio: 1.3716371834165084
average_variation: 0.3875261696400678
